In [5]:
import pandas as pd 

dataframe = pd.read_csv("gina_prior.csv")
dataframe.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3468 entries, 0 to 3467
Columns: 785 entries, pixel1 to label
dtypes: int64(785)
memory usage: 20.8 MB


In [6]:
dataframe['label'] = dataframe['label'].apply(lambda x : 0 if (x==-1) else x )

In [7]:
from sklearn.model_selection import train_test_split
X = dataframe.drop(columns=['label'])
y = dataframe['label']
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2)

In [8]:
# Launch tpot training 
from tpot import TPOTClassifier
clf_tpot = TPOTClassifier(verbosity=2, max_time_mins=30 )
clf_tpot.fit(X_train, y_train)

/home/datagenius/anaconda3/lib/python3.8/site-packages/tpot/builtins/__init__.py:36: UserWarning: Warning: optional dependency `torch` is not available. - skipping import of NN models.
  warnings.warn("Warning: optional dependency `torch` is not available. - skipping import of NN models.")



31.44 minutes have elapsed. TPOT will close down.
TPOT closed during evaluation in one generation.


TPOT closed prematurely. Will use the current best pipeline.

Best pipeline: ExtraTreesClassifier(input_matrix, bootstrap=False, criterion=entropy, max_features=0.9500000000000001, min_samples_leaf=4, min_samples_split=4, n_estimators=100)


TPOTClassifier(max_time_mins=30, verbosity=2)

In [9]:
import autosklearn.classification
clf_autosklearn = autosklearn.classification.AutoSklearnClassifier(time_left_for_this_task=1800)
clf_autosklearn.fit(X_train, y_train)

AutoSklearnClassifier(per_run_time_limit=180, time_left_for_this_task=1800)

In [10]:
from oboe import AutoLearner, error
import numpy as np 
xx_train = np.array(X_train)
yy_train = np.array(y_train)
method = 'Oboe' # 'Oboe' or 'TensorOboe'
problem_type = 'classification'
clf_oboe = AutoLearner(p_type=problem_type, runtime_limit=30, method=method, verbose=False)
clf_oboe.fit(xx_train, yy_train)

{'ranks': [8, 9],
 'runtime_limits': [2.0, 4.0],
 'validation_loss': [0.5, 0.16228776514977225, 0.08385864813877511],
 'filled_new_row': [array([[ 0.1778424 ,  0.16985914,  0.386646  ,  0.41758134,  0.40747532,
           0.16618454,  0.15531234,  0.37032305,  0.38813447,  0.38272652,
          -0.10112797, -0.09005342, -0.06911965, -0.04138537,  0.00933062,
           0.06488251,  0.11508523,  0.0564614 ,  0.20476992, -0.04143764,
          -0.06679475, -0.09982406,  0.15196187, -0.10112797, -0.05917865,
          -0.05421096, -0.06005868, -0.05208181, -0.02767001, -0.02638451,
           0.01137451,  0.01430449,  0.07449985,  0.07754043,  0.14116464,
           0.14286703,  0.16672182,  0.17072555,  0.07669558,  0.07711797,
          -0.06269687, -0.05907725,  0.00729785,  0.00913966,  0.18516544,
           0.19128358, -0.00522757, -0.00477809, -0.06344815, -0.05795696,
          -0.05917865, -0.05421096, -0.05917865, -0.05421096, -0.04233416,
           0.04367863, -0.087814  , -0.

In [11]:
from sklearn.metrics import accuracy_score
print("tpot score:\t\t ", clf_tpot.score(X_test, y_test))
print("autosklern score:\t\t ", clf_autosklearn.score(X_test, y_test))

xx_test = np.array(X_test)
yy_test = np.array(y_test)
y_predicted = clf_oboe.predict(xx_test)[0]
print("oboe score:\t\t ", accuracy_score(y_test, y_predicted) )

tpot score:		  0.9351585014409222
autosklern score:		  0.9582132564841499
oboe score:		  0.9178674351585014


In [12]:
tpot_pred    = clf_tpot.predict(X_test)
sklearn_pred = clf_autosklearn.predict(X_test)
oboe_pred    = clf_oboe.predict(X_test)[0]

In [15]:
prob = pd.DataFrame(list(zip(sklearn_pred, oboe_pred, tpot_pred)), columns=[ "AutoSklearn_Class", "Oboe_Class", "TPOT_Class"])
def function(x) :
    if [x['AutoSklearn_Class'],x['Oboe_Class'], x['TPOT_Class']].count(1)>=2 :
        return 1
    else   :
        return 0
    # else :
    #     int(x['AutoSklearn_Class'])
prob['ensemble'] = prob.apply(function, axis=1)

In [16]:
print(accuracy_score(y_test.astype(int), prob['ensemble'] ))

0.9423631123919308
